# 第５ステージ　DeZeroで挑む

## ステップ52　GPU対応

### 52.1 - 52.4
省略

### 52.5　GPUでMNISTを学習

In [1]:
import time
import leopard
import leopard.functions as F
from leopard import optimizers
from leopard import DataLoader
from leopard.models import MLP

In [2]:
max_epoch = 5
batch_size = 100

train_set = leopard.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

In [3]:
# GPU mode
if leopard.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()

In [4]:
for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9084, time: 2.1086[sec]
epoch: 2, loss: 1.2747, time: 1.8774[sec]
epoch: 3, loss: 0.9189, time: 1.8558[sec]
epoch: 4, loss: 0.7362, time: 1.8578[sec]
epoch: 5, loss: 0.6323, time: 1.8633[sec]


試しにCPUモードで学習してみる。

In [5]:
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

In [6]:
train_loader.to_cpu()
model.to_cpu()

In [7]:
for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9127, time: 4.4109[sec]
epoch: 2, loss: 1.2858, time: 4.4089[sec]
epoch: 3, loss: 0.9272, time: 4.3324[sec]
epoch: 4, loss: 0.7422, time: 4.0596[sec]
epoch: 5, loss: 0.6371, time: 4.0639[sec]


## ステップ53 モデルの保存と読み込み

### 53.1 NumPyのsave関数とload関数
省略

### 53.2 Layerクラスのパラメータをフラットに

In [8]:
import numpy as np

from leopard import Layer
from leopard import Parameter

In [9]:
layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

In [10]:
params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)

{'l1/p1': variable(1), 'p3': variable(3), 'p2': variable(2)}


### 53.3 Layerクラスのsave関数とload関数

In [11]:
import os

In [12]:
max_epoch = 3
batch_size = 100

train_set = leopard.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

In [13]:
# パラメータの読み込み
if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')

In [14]:
for epoch in range(max_epoch):
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    print('epoch: {}, loss: {:.4f}'.format(
        epoch + 1, sum_loss / len(train_set)))

# パラメータの保存
model.save_weights('my_mlp.npz')

epoch: 1, loss: 0.5194
epoch: 2, loss: 0.4867
epoch: 3, loss: 0.4620


## ステップ54　Dropoutとテストモード
例えば、10個のニューロンからなる層があり、その層の次にDropoutレイヤを用いて60%のニューロンをランダムで消去する。つまり、毎回平均して４つの出力だけが次の層へ伝達し、残りは０になる。

In [15]:
dropout_ratio  = 0.6
x = np.ones(10)

mask = np.random.rand(10) > dropout_ratio
print(mask)

y = x * mask
print(y)

[ True False False  True  True  True  True  True False False]
[1. 0. 0. 1. 1. 1. 1. 1. 0. 0.]


推論時には、全てのニューロンを使いながら、学習時の挙動を**真似る**必要がある。それには、全てのニューロンを使って計算し、その出力を**弱める**ことで対応できる。<br>

先ほどの例では、学習時平均して40%のニューロンが使用される。それを考慮して、推論時には、全てのニューロンを使って計算し、出力を0.4倍する。

In [16]:
# 学習時
mask = np.random.rand(*x.shape) > dropout_ratio
y = x * mask

# 推論時
scale = 1 - dropout_ratio
y = x * scale

### 54.2　Inverted Dropout
先ほど、**推論時**に`scale`を乗算した。ここで、推論時に何も行わないように、学習時に先回りしてニューロンの値を`1 / scale`倍する。そうすることで主に以下の２つのような利点がある。

1. 推論時の処理速度が少し向上する
2. dropout_ratioを動的に変更できる  

In [17]:
# 学習時
scale = 1 - dropout_ratio
mask = np.random.rand(*x.shape) > dropout_ratio
y = x * mask / scale

# 推論時
y = x

### 54.3 テストモードの追加
Dropoutを使うには、学習と推論のフェーズを判別する必要がある。そこで、逆伝播が不要なモード(`with leopard.no_grad():`)の仕組みを流用する。

### 54.4　Dropoutの実装

In [20]:
from leopard import test_mode

x = np.ones(5)
print(x)

# 学習時
y = F.dropout(x)
print(y)

# 推論時
with test_mode():
    y = F.dropout(x)
    print(y)

[1. 1. 1. 1. 1.]
variable([2. 0. 2. 0. 0.])
variable([1. 1. 1. 1. 1.])
